# Thermal Model Analysis

This notebook runs the thermal model analysis using the `Code.py` script.

In [ ]:
# Import necessary libraries
import numpy as np

# Run the Code.py script and handle errors
try:
    %run Code.py
except FileNotFoundError:
    print('Error: Code.py script not found.')
except Exception as e:
    print(f'Error while running Code.py: {e}')

# Ensure the necessary variables are defined
try:
    A
    G
    C
    b
    f
    θ
    q
    As
    Bs
    Cs
    Ds
except NameError as e:
    print(f'Error: {e}')

## Connectivity Matrix (A)

In [ ]:
print("A matrix (connectivity):")
try:
    print(A)
except NameError:
    print('A matrix is not defined.')

## Conductance Matrix (G)

In [ ]:
print("\nG matrix (conductance):")
try:
    print(G)
except NameError:
    print('G matrix is not defined.')

## Capacitance Matrix (C)

In [ ]:
print("\nC matrix (capacitances):")
try:
    print(C)
except NameError:
    print('C matrix is not defined.')

## External Temperatures and Sources Matrix (b)

In [ ]:
print("\nb matrix (external temperatures and sources):")
try:
    print(b)
except NameError:
    print('b matrix is not defined.')

## Absorbed Solar Radiation and Internal Sources Matrix (f)

In [ ]:
print("\nf matrix (absorbed solar radiation and internal sources):")
try:
    print(f)
except NameError:
    print('f matrix is not defined.')

## Indoor Air Temperatures

In [ ]:
print("\nIndoor air temperatures:")
try:
    indoor_air = [3, 7]  # Adjust indices if needed
    print(f"Room A temperature: {θ[3]} °C")
    print(f"Room B temperature: {θ[7]} °C")
except NameError:
    print('θ is not defined.')
except IndexError as e:
    print(f'Error: {e}')

## Controller Thermal Load (Room B)

In [ ]:
try:
    controller = 7  # Assuming the controller is defined at index 7, update as needed
    print(f"\nController thermal load (Room B): {q[controller]} W")
except NameError:
    print('q or controller is not defined.')
except IndexError as e:
    print(f'Error: {e}')

## State-Space Representation Matrices

In [ ]:
print("\nState-space representation matrices:")
try:
    print("As:\n", As)
    print("Bs:\n", Bs)
    print("Cs:\n", Cs)
    print("Ds:\n", Ds)
except NameError as e:
    print(f'Error: {e}')

## Step Response of Indoor Air Temperature

In [ ]:
# Define the time parameters
dt = 1.0  # Reduced time step for better stability
total_time = 24 * 3600  # Total simulation time in seconds (24 hours)
time_steps = np.arange(0, total_time, dt)

# Initialize temperature array
theta = np.zeros((len(time_steps), len(θ)))
theta[0, :] = θ  # Initial condition

# Apply a step change in outdoor temperature at t = 0
To_step = 0.0  # New outdoor temperature after the step change
b[[0, 1, 14]] = To_step

# Regularization term to stabilize As
regularization_term = 1e-5
As_reg = As - regularization_term * np.eye(len(θ))

# Function to perform backward euler integration step
def backward_euler_step(theta_prev, As_reg, Bs, dt):
    I = np.eye(len(As_reg))
    theta_dot = np.linalg.inv(I - dt * As_reg) @ (theta_prev + dt * Bs.flatten())
    return theta_dot

# Calculate the step response
for i in range(1, len(time_steps)):
    try:
        theta[i, :] = backward_euler_step(theta[i-1, :], As_reg, Bs, dt)
        if np.any(np.abs(theta[i, :]) > 1e10):
            raise OverflowError("Potential overflow detected in theta.")
    except OverflowError as e:
        print(f"Overflow detected at step {i}. Clamping values.")
        theta[i, :] = np.clip(theta[i, :], -1e10, 1e10)  # Clamp values to prevent overflow

# Plot the step response
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(time_steps / 3600, theta[:, indoor_air[0]], label='Room A Air Temperature (Node 3)')
plt.plot(time_steps / 3600, theta[:, indoor_air[1]], label='Room B Air Temperature (Node 7)')
plt.axhline(y=To_step, color='r', linestyle='--', label='Setpoint Temperature')
plt.xlabel('Time (hours)')
plt.ylabel('Temperature (°C)')
plt.title('Step Response of Indoor Air Temperature to Outdoor Temperature Change')
plt.legend()
plt.grid(True)
plt.show()